In [ ]:
from enum import Enum


class Color(Enum):
    """
    Enum for color codes.
    """

    BLACK = 0
    RED = 1
    GREEN = 2
    YELLOW = 3


class Size(Enum):
    """
    Enum for size codes.
    """

    SMALL = 0
    MEDIUM = 1
    LARGE = 2


class Product:
    """
    Class representing a product with color and size attributes.
    """

    def __init__(self, name, color : Color, size: Size):
        self.name = name
        self.color = color
        self.size = size


# this class violates the Open/Closed Principle
# because it requires modification to add new filters
class ProductFilter:
    """
    Class to filter products based on color and size.
    """

    def __init__(self, products):
        self.products = products

    def filter_by_color(self, color: Color):
        return [product for product in self.products if product.color == color]

    def filter_by_size(self, size: Size):
        return [product for product in self.products if product.size == size]
    
    def filter_by_color_and_size(self, color: Color, size: Size):
        return [product for product in self.products if product.color == color and product.size == size]

In [ ]:
# Solution - Implementation of Specification and Filter by following the Specification Enterprise Pattern
class Specification:
    """
    Interface for specifications.
    """

    def is_satisfied(self, item):
        # you are meant to override this method as the whole idea of the OCP principle is that you extend the code
        pass


class Filter:
    """
    Interface for filters.
    """

    def filter(self, items: list, specification: Specification):
        # you are meant to override this method as the whole idea of the OCP principle is that you extend the code
        pass


class ColorSpecification(Specification):
    """
    Specification for filtering products by color.
    """

    def __init__(self, color: Color):
        self.color = color

    def is_satisfied(self, item: Product):
        return item.color == self.color


class SizeSpecification(Specification):
    """
    Specification for filtering products by size.
    """

    def __init__(self, size: Size):
        self.size = size

    def is_satisfied(self, item: Product):
        return item.size == self.size

# Combinator
class AndSpecification(Specification):
    """
    Specification for combining multiple specifications with AND logic.
    """

    def __init__(self, *specifications):
        self.specifications = specifications

    def is_satisfied(self, item: Product):
        return all(map(lambda spec: spec.is_satisfied(item), self.specifications))


class BetterFilter(Filter):
    """
    Class to filter products based on specifications.
    """

    def filter(self, items: list, specification: Specification):
        return [item for item in items if specification.is_satisfied(item)]
    
    
# Example usage
if __name__ == "__main__":
    apple = Product("Apple", Color.GREEN, Size.SMALL)
    tree = Product("Tree", Color.GREEN, Size.LARGE)
    house = Product("House", Color.RED, Size.MEDIUM)

    products = [apple, tree, house]

    # Using the original filter - this violates the Open/Closed Principle
    pf = ProductFilter(products)
    green_products = pf.filter_by_color(Color.GREEN)
    print(f"Green products: {[product.name for product in green_products]}")

    # Using the new filter with specifications - this adheres to the Open/Closed Principle
    bf = BetterFilter()
    green_spec = ColorSpecification(Color.GREEN)
    small_spec = SizeSpecification(Size.SMALL)
    green_and_small_spec = AndSpecification(green_spec, small_spec)

    filtered_products = bf.filter(products, green_and_small_spec)
    print(f"Filtered products: {[product.name for product in filtered_products]}")